In [42]:
import vectorbt as vbt
import pandas as pd 
import numpy as np
from datetime import datetime
from kucoin_candle_spot import SpotDataFetcher
from datetime import datetime, timezone
import pandas_ta as ta

from numba import njit
import plotly.graph_objs as go
from vectorbt.portfolio.enums import Direction, SizeType
import talib



## data collection

In [43]:
symbols = [
    "SOL-USDT",   # Solana
]
timeframe = "1day"
start_time = "2018-01-01 10:00:00"
end_time = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")


# Fetch data for each symbol and store in a dictionary

dataframes = {}
for symbol in symbols:
    try:
        fetcher = SpotDataFetcher(symbol, timeframe, start_time, end_time)
        df = fetcher.fetch_candles_as_df()
        df['symbol'] = symbol
        dataframes[symbol] = df
    except Exception as e:
        print(f"Error fetching data: {e}")

symbols = list(dataframes.keys())



INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetching candle data...


INFO:kucoin_candle_spot.kucoin_fetch_spot:Fetch complete. Chunks: 2, Candles: 1300


In [44]:
symbols = list(dataframes.keys())
symbols

['SOL-USDT']

In [45]:

def momentum_strategy(close, high, low, atr_length, ema_length, ema2_length, vola_multiplier, vol_window, symbol=None):
    # Calculate EMA
    EMA = vbt.IndicatorFactory.from_talib('EMA')
    ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()
    ema2 = EMA.run(close, timeperiod=ema2_length).real.to_numpy()

    # Calculate ATR
    ATR = vbt.IndicatorFactory.from_talib('ATR')
    atr = ATR.run(high, low, close, timeperiod=atr_length).real.to_numpy()

    # Function to calculate rolling max
    def rolling_max(arr, window):
        result = np.full_like(arr, np.nan)
        for i in range(window - 1, len(arr)):
            result[i] = np.max(arr[i - window + 1:i + 1])
        return result

    # Function to compute bearish volatility
    def compute_is_bearish_vol(high, low, atr, window):
        rm = rolling_max(high, window=window)
        return (rm - low) > (atr * vola_multiplier)

    # Calculate conditions
    is_bullish = (close > ema) & (close > ema2)
    is_bearish_vol = compute_is_bearish_vol(high, low, atr, window=vol_window)
    is_caution = is_bullish & (is_bearish_vol | (close < ema))
    signal_buy = is_bullish & ~is_caution

    # Define conditions and corresponding values
    conditions = [signal_buy, is_caution]
    values = [10, 5]

    # Create final signal using np.select
    signal = np.select(conditions, values, default=0)

    return signal.reshape(close.shape)

# Create the indicator factory
momentum_indicator = vbt.IndicatorFactory(
    class_name='MomentumStrategy',
    short_name='momentum',
    input_names=['Close', 'High', 'Low'],
    param_names=['atr_length', 'ema_length', 'ema2_length', 'vola_multiplier', 'vol_window','symbol'],
    output_names=['signal']
).from_apply_func(momentum_strategy)


In [46]:

# Define parameter ranges
atr_length = np.arange(5, 8, 1)  # Range from 5 to 7 with step 1
ema_length = np.arange(100, 241, 20)  # Range from 100 to 240 with step 20
ema2_length = np.arange(10, 50, 5)  # Range from 10 to 45 with step 5
vola_multiplier = [1.6, 1.7, 1.8]  # Specific values for volatility multiplier
vol_window = np.arange(7, 10, 1)  # Range from 7 to 9 with step 1

# # # Define parameter ranges
# atr_length = 7# Range from 5 to 7 with step 1
# ema_length = 200  # Range from 100 to 240 with step 20
# ema2_length = 50  # Range from 10 to 45 with step 5
# vola_multiplier = 1.5 # Specific values for volatility multiplier
# vol_window = np.arange(7, 10, 1)  # Range from 7 to 9 with step 1

indicator_obj = {}

for symbol in symbols:
    # Run the indicator
    df_pair = dataframes[symbol]

    indicator = momentum_indicator.run(
        df_pair['close'], df_pair['high'], df_pair['low'],
        atr_length=atr_length,
        ema_length=ema_length,
        ema2_length=ema2_length,
        vola_multiplier=vola_multiplier,
        vol_window=vol_window,
        symbol= symbol,
        param_product=True
    )
    indicator_obj[symbol] = indicator

indicator_obj

{'SOL-USDT': <vectorbt.indicators.factory.MomentumStrategy at 0x12627ecd0>}

In [47]:
def copy_rename_close_series(df_original, df_indicator_signals):

    def repeat_series_horizontally(series, target_shape):
        return np.tile(series.values.reshape(-1, 1), target_shape[1])

    # Assuming df and df_indicator_signals are already defined
    nedded_copies = len(df_indicator_signals.columns)

    # Repeat the 'close' series horizontally to match the shape of df_indicator_signals
    df_dublicate_close = pd.DataFrame(repeat_series_horizontally(df_original['close'], df_indicator_signals.shape), index=df_original.index)

    # Rename columns if shapes match
    # if df_indicator_signals.shape == df_dublicate_close.shape:
    df_dublicate_close.columns = df_indicator_signals.columns
    #     print("Columns have been renamed")
    # else:
    #     print("The DataFrames do not have the same shape.")
    
    
    return df_dublicate_close



df_dublicate_close_pairs ={}
for symbol in symbols:
    print(symbol)
    df = dataframes[symbol]
    df_dublicate_close = copy_rename_close_series(df, indicator_obj[symbol].signal)
    print('New Schape',df_dublicate_close.shape)
    df_dublicate_close_pairs[symbol] = df_dublicate_close
    # df_dublicate_close_pairs.append(df_dublicate_close)

print(df_dublicate_close_pairs.keys())






SOL-USDT
New Schape (1300, 1728)
dict_keys(['SOL-USDT'])


In [48]:
# add choosing cutoff time for backtesting
# create seperation of data for backtesting and validation 


missing_data_pairs = []
for symbol in symbols:
    df_check = dataframes[symbol]
    # Create a complete date range based on the start and end of your DataFrame
    complete_date_range = pd.date_range(start=df_check.index.min(), end=df_check.index.max(), freq='1D')

    # Check for missing dates
    missing_dates = complete_date_range.difference(df_check.index)

    if not missing_dates.empty:
        # print('missing data in:',symbol)
        # print("Missing dates:")
        # print(missing_dates)
        missing_data_pairs.append(symbol)
    # else:
         # print("All dates are accounted for.")
    
missing_data_pairs


[]

In [49]:
# df_dublicate_close_pairs['SOL-USDT']

In [50]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,tp_hit,entry_price,atr_values):
    close_price = c.close[c.i, c.col]

    # ====================== Active Position ======================
    if (c.position_now > 0):

        # ------------------------- Exit Position -------------------------

        if open_[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # sell all position (be aware of the negative sign)

                price=open_[c.i],  

                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)


            return value
        
        # ------------------------- Take profit ----------------------
        
        # # if (tp_prices[c.i] < high[c.i]) & (tp_hit[c.i] == False):
        # if (high[c.i] >= tp_prices[c.i]) and (not tp_hit[c.i]):

        #     tp_hit[:] = True

        #     value_sell = vbt.portfolio.nb.order_nb(
        #         size= -0.5,  # tp hit sell 50% of position

        #         price=tp_prices[c.i],  # take profit at tp price 

        #         size_type=SizeType.Percent,
        #         direction=Direction.LongOnly,

        #         fees=0.001,
        #         slippage=0.002)
            
        #     return value_sell
        
        
        # ------------------------- Trailing SL Update ----------------------

        # sl update for long position
        if (entries[c.i-1, c.col] == 5):  
            highest_low = np.max(low[c.i-7:c.i])
            update2 = highest_low - atr_values[c.i] * 0.2
            if update2 > sl_prices[c.i]:
                sl_prices[:]= update2

        #losen sl for less volatility 
        if (entries[c.i-1, c.col] == 10): 
            highest_low = np.max(low[c.i-7:c.i])
            update = highest_low - atr_values[c.i]
            if update > sl_prices[c.i]:
                sl_prices[:]= update



    
    # ====================== Entry Position ======================
  

    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i-1, c.col] == 10:

            # reset values to default 
            sl_prices[:] = np.nan
            entry_price[:] = open_[c.i]
            tp_hit[:] = False

            order = vbt.portfolio.nb.order_nb(
                size=1,  # Adjusted order size

                price=open_[c.i],  # Current closing price
                
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.001,  # No fees
                slippage=0.002,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )

            # set intial stop loss
            sl_prices[:] = low[c.i] - atr_values[c.i] 

            # set take profit price
            atr_multiple = 1
            tp_prices[:] = entry_price[:] + (atr_values[c.i] * atr_multiple)

            return order

    
    return vbt.portfolio.enums.NoOrder


In [51]:
pf_pairs = {}

for symbol in symbols:
    # candle data
    df = dataframes[symbol]
    close = df_dublicate_close_pairs[symbol]
    open_ = df['open'].to_numpy().flatten()
    high = df['high'].to_numpy().flatten()
    low = df['low'].to_numpy().flatten()

    entries = indicator_obj[symbol].signal.values

    atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()

    # Create an array to store SL prices
    sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
    tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
    entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array
    tp_hit = np.full(close.shape[0], False)  # Use a 1D array


    # Create portfolio
    pf = vbt.Portfolio.from_order_func(
        close,           # Price DataFrame
        order_func_nb,
        high,
        low,
        open_,
        entries,    # Order function
        sl_prices,
        tp_prices,
        tp_hit,
        entry_price,
        atr_values,  # Pass the SL prices array
        init_cash=500  # Initial cash balance
    )

    pf_pairs[symbol] = pf


In [52]:
# indicator_obj['SOL-USDT'].signal.columns



In [53]:
# Combine the total return and max drawdown for each symbol
first_glance = []

for symbol in symbols:
    # if symbol not in missing_data_pairs:
    total_return = pf_pairs[symbol].total_return()
    max_dd = pf_pairs[symbol].max_drawdown()
    sharpe_ratio = pf_pairs[symbol].sharpe_ratio()
    sortino_ratio = pf_pairs[symbol].sortino_ratio()
    beta = pf_pairs[symbol].beta()
    profit_factor = pf_pairs[symbol].trades.profit_factor()
    win_rate = pf_pairs[symbol].trades.win_rate()
    total_trades = pf_pairs[symbol].trades.count()
        
        # return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
        # return_and_maxdd = pd.concat([total_return, max_dd,total_trades,win_rate,profit_factor], axis=1)

    return_and_maxdd = pd.concat([total_return, max_dd,total_trades,sharpe_ratio,sortino_ratio,win_rate,beta,profit_factor], axis=1)


    first_glance.append(return_and_maxdd)

# Concatenate all DataFrames along the rows
combined_df = pd.concat(first_glance, axis=0)

combined_df

total_return  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol                 
5                   100                 10                   1.6                      7                   SOL-USDT             1.355743   
                                                                                      8                   SOL-USDT             1.350063   
                                                                                      9                   SOL-USDT             1.296828   
                                                             1.7                      7                   SOL-USDT             1.454651   
                                                                                      8                   SOL-USDT             1.483131   
...                                                                                                                                 ...   
7                   240                 45                   1.7                      8                   SOL-USDT             1.624902   
                                                                                      9                   SOL-USDT             2.024373   
                                                             1.8                      7                   SOL-USDT             1.176277   
                                                                                      8                   SOL-USDT             2.346738   
                                                                                      9                   SOL-USDT             2.458384   

                                                                                                                           max_drawdown  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol                 
5                   100                 10                   1.6                      7                   SOL-USDT            -0.462417   
                                                                                      8                   SOL-USDT            -0.469864   
                                                                                      9                   SOL-USDT            -0.510265   
                                                             1.7                      7                   SOL-USDT            -0.501726   
                                                                                      8                   SOL-USDT            -0.501726   
...                                                                                                                                 ...   
7                   240                 45                   1.7                      8                   SOL-USDT            -0.395889   
                                                                                      9                   SOL-USDT            -0.377721   
                                                             1.8                      7                   SOL-USDT            -0.536752   
                                                                                      8                   SOL-USDT            -0.367188   
                                                                                      9                   SOL-USDT            -0.373861   

                                                                                                                           count  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol          
5                   100                 10                   1.6                      7                   SOL-USDT            38   
                                                                                      8                   SOL-U

In [54]:
profit_factor_check = combined_df.sort_values(by=['profit_factor'], ascending=False)
profit_factor_check.head(50)

total_return  \
momentum_atr_length momentum_ema_length momentum_ema2_length momentum_vola_multiplier momentum_vol_window momentum_symbol                 
5                   240                 40                   1.8                      9                   SOL-USDT             3.555599   
                                        25                   1.7                      8                   SOL-USDT             3.546296   
6                   240                 10                   1.7                      9                   SOL-USDT             3.620149   
                                        25                   1.7                      8                   SOL-USDT             3.401357   
5                   240                 45                   1.8                      9                   SOL-USDT             3.426984   
                                        25                   1.7                      9                   SOL-USDT             3.294896   
                                        35                   1.8                      9                   SOL-USDT             3.391361   
                                        10                   1.6                      9                   SOL-USDT             3.313562   
6                   240                 25                   1.7                      9                   SOL-USDT             3.157972   
                                        10                   1.7                      8                   SOL-USDT             3.366977   
5                   240                 10                   1.7                      9                   SOL-USDT             3.425204   
                    180                 40                   1.8                      9                   SOL-USDT             3.064305   
                                        25                   1.7                      8                   SOL-USDT             3.131632   
                    220                 25                   1.7                      8                   SOL-USDT             2.753163   
                    200                 40                   1.8                      9                   SOL-USDT             2.731253   
                                        25                   1.7                      8                   SOL-USDT             2.793063   
                    240                 25                   1.6                      8                   SOL-USDT             3.094265   
                    220                 40                   1.8                      9                   SOL-USDT             2.646486   
6                   180                 10                   1.7                      9                   SOL-USDT             3.061667   
                                        25                   1.7                      8                   SOL-USDT             2.999913   
                    220                 10                   1.7                      9                   SOL-USDT             2.706704   
                    240                 10                   1.7                      7                   SOL-USDT             3.316889   
                    220                 25                   1.7                      8                   SOL-USDT             2.633510   
                    200                 10                   1.7                      9                   SOL-USDT             2.728831   
                                        25                   1.7                      8                   SOL-USDT             2.672137   
5                   240                 35                   1.7                      9                   SOL-USDT             3.074486   
                                        40                   1.7                      9                   SOL-USDT             3.074486   
                    180                 45                   1.8                      9     

In [55]:
index = 1

profit_factor_check
test_df = profit_factor_check.copy()
test_df=test_df.reset_index()
test_df.iloc[index]
total_return_index = test_df.columns.get_loc('total_return')
total_return_index
name_of_setup_to_investigate = tuple(test_df.iloc[index, :total_return_index ])
name_of_setup_to_investigate

(5, 240, 25, 1.7, 8, 'SOL-USDT')

In [56]:
# index = 1

# combined_df.sort_values(by='total_return', ascending=False).head(50)
# combined_df
# test_df = combined_df.copy()
# test_df=test_df.reset_index()
# test_df.iloc[index]
# total_return_index = test_df.columns.get_loc('total_return')
# total_return_index
# name_of_setup_to_investigate = tuple(test_df.iloc[index, :total_return_index ])
# name_of_setup_to_investigate

In [57]:
name_of_setup_to_investigate[-1]

'SOL-USDT'

In [58]:
# pf_pairs[name_of_setup_to_investigate[-1]][name_of_setup_to_investigate[1:]].plot().show()
pf_pairs[name_of_setup_to_investigate[-1]][name_of_setup_to_investigate].plot().show()

In [59]:
pf_pairs['SOL-USDT'][name_of_setup_to_investigate].orders.records_readable


,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,35742,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-07-28 00:00:00+00:00,19.865446,25.144188,0.499500,Buy
1,35743,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-08-03 00:00:00+00:00,19.865446,23.096714,0.458827,Sell
2,35744,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-08-10 00:00:00+00:00,18.743163,24.430764,0.457910,Buy
3,35745,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-08-17 00:00:00+00:00,18.743163,22.773362,0.426845,Sell
4,35746,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-10-03 00:00:00+00:00,18.184769,23.425758,0.425992,Buy
5,35747,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-10-12 00:00:00+00:00,18.184769,21.967976,0.399483,Sell
6,35748,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-10-18 00:00:00+00:00,16.625104,23.980866,0.398684,Buy
7,35749,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-11-22 00:00:00+00:00,16.625104,51.558676,0.857168,Sell
8,35750,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-11-25 00:00:00+00:00,15.023637,56.940654,0.855456,Buy
9,35751,"(5, 240, 25, 1.7, 8, SOL-USDT)",2023-12-13 00:00:00+00:00,15.023637,68.435854,1.028155,Sell
